In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

from category_encoders.ordinal import OrdinalEncoder

from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
houses = pd.read_csv('../Data and Notes/for_preproc.csv')
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455 entries, 0 to 1454
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1455 non-null   int64  
 1   MSSubClass     1455 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1198 non-null   float64
 4   LotArea        1455 non-null   int64  
 5   Street         1455 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1455 non-null   object 
 8   LandContour    1455 non-null   object 
 9   Utilities      1455 non-null   object 
 10  LotConfig      1455 non-null   object 
 11  LandSlope      1455 non-null   object 
 12  Neighborhood   1455 non-null   object 
 13  Condition1     1455 non-null   object 
 14  Condition2     1455 non-null   object 
 15  BldgType       1455 non-null   object 
 16  HouseStyle     1455 non-null   object 
 17  OverallQual    1455 non-null   int64  
 18  OverallC

In [3]:
X = houses.drop(columns=['SalePrice', 'Id'])
y = houses['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")

Train set size: (1164, 79)
Validation set size: (291, 79)


In [4]:
# Target distribution is skewed, log to normalize distribution

log_y_train = np.log(y_train)
log_y_val = np.log(y_val)

In [5]:
# Transformers to use

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')
na_imputer = SimpleImputer(strategy='constant', fill_value='NA')
zero_imputer = SimpleImputer(strategy='constant', fill_value= 0)
scaler = StandardScaler()


In [6]:
# Ordinal Map

dict_ex_to_po = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
dict_func = {'Typ':8,	'Min1':7,	'Min2':6,	'Mod':5, 'Maj1':4, 'Maj2':3, 'Sev':2,	'Sal':1}

ordinal_map = [
  {'col': 'ExterQual','mapping':dict_ex_to_po},
  {'col': 'ExterCond','mapping':dict_ex_to_po},
  {'col': 'HeatingQC','mapping':dict_ex_to_po},
  {'col': 'KitchenQual','mapping':dict_ex_to_po},
  {'col': 'Functional','mapping':dict_func}
]

ord = OrdinalEncoder(mapping = ordinal_map)

In [7]:
# Column lists for use in pipeline

na_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType',
               'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MasVnrType', 'MiscFeature',
               'MSSubClass','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1',
               'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','Foundation','Heating','CentralAir',
               'PavedDrive','SaleType','SaleCondition'
               ]

mode_cols = ['Electrical']

zero_cols = ['MasVnrArea']

median_cols = ['LotFrontage']

ohe_cols = [col for col in (na_cols + mode_cols) if col != 'Id']

ord_cols = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual', 'Functional']

num_remove = ['Id', 'MSSubClass', 'SalePrice', 'GarageYrBlt','MasVnrArea']

nums_cols = list(houses.select_dtypes('number').columns.difference(num_remove))

scale_cols = ord_cols + nums_cols

In [8]:
# Convert int to string for NA imputation

def convert_to_string(df):
  df= df.copy()
  df= df.astype(str)
  return df

string_convert = FunctionTransformer(convert_to_string)


In [9]:
# Function to add calculated columns

def calc_cols(df):
  df = df.copy()
  df['AreaRatio'] = df['GrLivArea'] / df['LotArea']
  df['BedBath'] = (df['FullBath'] + df['HalfBath']) / (df['BedroomAbvGr'] + 1)
  return df

add_cols = FunctionTransformer(calc_cols)

In [10]:
# String conversion object for pipeline

proc_retype = ColumnTransformer([
  ('retype', string_convert, ['GarageYrBlt', 'MSSubClass'])
],remainder='passthrough', verbose_feature_names_out=False)


In [11]:
# Imputer object for pipeline

proc_impute = ColumnTransformer([
  ('median_imputer', median_imputer, nums_cols),
  ('na_imputer', na_imputer, na_cols),
  ('freq_imputer', freq_imputer, mode_cols),
  ('zero_imputer', zero_imputer, zero_cols)
],remainder='passthrough', verbose_feature_names_out=False)

In [12]:
#Encoder object for pipeline

proc_encode = ColumnTransformer([
  ('ohe', ohe, ohe_cols),
  ('ord', ord, ord_cols),
], remainder='passthrough', verbose_feature_names_out=False)

In [13]:
#Scaling object for pipeline

proc_scale = ColumnTransformer([
  ('standard_scaler', StandardScaler(), scale_cols)
], remainder='passthrough', verbose_feature_names_out=False)

In [14]:
# Added features object for pipeline

proc_add_col = ColumnTransformer([
  ('add_col', add_cols, ['FullBath', 'HalfBath', 'BedroomAbvGr', 'GrLivArea', 'LotArea'])
], remainder='passthrough', verbose_feature_names_out=False)

In [15]:
# Create pipeline

main_pipe = Pipeline([
  ('proc_retype', proc_retype),
  ('proc_impute', proc_impute),
  ('proc_add_col', proc_add_col),
  ('proc_encode', proc_encode),
  ('proc_scale', proc_scale)
])

In [16]:
# Check x train

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1164 entries, 720 to 1126
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1164 non-null   int64  
 1   MSZoning       1164 non-null   object 
 2   LotFrontage    948 non-null    float64
 3   LotArea        1164 non-null   int64  
 4   Street         1164 non-null   object 
 5   Alley          74 non-null     object 
 6   LotShape       1164 non-null   object 
 7   LandContour    1164 non-null   object 
 8   Utilities      1164 non-null   object 
 9   LotConfig      1164 non-null   object 
 10  LandSlope      1164 non-null   object 
 11  Neighborhood   1164 non-null   object 
 12  Condition1     1164 non-null   object 
 13  Condition2     1164 non-null   object 
 14  BldgType       1164 non-null   object 
 15  HouseStyle     1164 non-null   object 
 16  OverallQual    1164 non-null   int64  
 17  OverallCond    1164 non-null   int64  
 18  YearBuilt  

In [17]:
# Fit pipeline on train data

main_pipe.fit(X_train)

c:\Users\nick_\Desktop\Learning Fuze\Bootcamp\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('proc_retype',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('retype',
                                                  FunctionTransformer(func=<function convert_to_string at 0x0000023E48412FC0>),
                                                  ['GarageYrBlt',
                                                   'MSSubClass'])],
                                   verbose_feature_names_out=False)),
                ('proc_impute',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('median_imputer',
                                                  SimpleImputer(strategy='m...
                                                   '2ndFlrSF', '3SsnPorch',
                                                   'BedroomAbvGr', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'BsmtUnfSF',
                                                   'EnclosedPorch',
                                                   'Fireplaces', 'FullBath',
                                                   'GarageArea', 'GarageCars',
                                                   'GrLivArea', 'HalfBath',
                                                   'KitchenAbvGr', 'LotArea',
                                                   'LotFrontage',
                                                   'LowQualFinSF', 'MiscVal',
                                                   'MoSold', 'OpenPorchSF',
                                                   'OverallCond', 'OverallQual', ...])],
                                   verbose_feature_names_out=False))])

In [18]:
# Transform train and val from fit

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

In [19]:
# Check processed train

X_train_proc.head()

,ExterQual,ExterCond,HeatingQC,KitchenQual,Functional,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,Alley_Grvl,Alley_NA,Alley_Pave,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NA,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NA,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NA,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NA,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NA,BsmtFinType2_Rec,BsmtFinType2_Unf,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NA,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageYrBlt_1900.0,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935.0,GarageYrBlt_1936.0,GarageYrBlt_1937.0,GarageYrBlt_1938.0,GarageYrBlt_1939.0,GarageYrBlt_1940.0,GarageYrBlt_1941.0,GarageYrBlt_1942.0,GarageYrBlt_1945.0,GarageYrBlt_1946.0,GarageYrBlt_1948.0,GarageYrBlt_1949.0,GarageYrBlt_1950.0,GarageYrBlt_1951.0,GarageYrBlt_1952.0,GarageYrBlt_1953.0,GarageYrBlt_1954.0,GarageYrBlt_1955.0,GarageYrBlt_1956.0,GarageYrBlt_1957.0,GarageYrBlt_1958.0,GarageYrBlt_1959.0,GarageYrBlt_1960.0,GarageYrBlt_1961.0,GarageYrBlt_1962.0,GarageYrBlt_1963.0,GarageYrBlt_1964.0,GarageYrBlt_1965.0,GarageYrBlt_1966.0,GarageYrBlt_1967.0,GarageYrBlt_1968.0,GarageYrBlt_1969.0,GarageYrBlt_1970.0,GarageYrBlt_1971.0,GarageYrBlt_1972.0,GarageYrBlt_1973.0,GarageYrBlt_1974.0,GarageYrBlt_1975.0,GarageYrBlt_1976.0,GarageYrBlt_1977.0,GarageYrBlt_1978.0,GarageYrBlt_1979.0,GarageYrBlt_1980.0,GarageYrBlt_1981.0,GarageYrBlt_1982.0,GarageYrBlt_1983.0,GarageYrBlt_1984.0,GarageYrBlt_1985.0,GarageYrBlt_1986.0,GarageYrBlt_1987.0,GarageYrBlt_1988.0,GarageYrBlt_1989.0,GarageYrBlt_1990.0,GarageYrBlt_1991.0,GarageYrBlt_1992.0,GarageYrBlt_1993.0,GarageYrBlt_1994.0,GarageYrBlt_1995.0,GarageYrBlt_1996.0,GarageYrBlt_1997.0,GarageYrBlt_1998.0,GarageYrBlt_1999.0,GarageYrBlt_2000.0,GarageYrBlt_2001.0,GarageYrBlt_2002.0,GarageYrBlt_2003.0,GarageYrBlt_2004.0,GarageYrBlt_2005.0,GarageYrBlt_2006.0,GarageYrBlt_2007.0,GarageYrBlt_2008.0,GarageYrBlt_2009.0,GarageYrBlt_2010.0,GarageYrBlt_nan,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_NA,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NA,GarageCond_Po,GarageCond_TA,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_NA,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_NA,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighbor

In [20]:
# Check processed train set for nulls

X_train_proc.isnull().sum().sort_values(ascending=False)

MasVnrArea        0
ExterQual         0
ExterCond         0
HeatingQC         0
SaleType_ConLw    0
                 ..
BedroomAbvGr      0
3SsnPorch         0
2ndFlrSF          0
1stFlrSF          0
Functional        0
Length: 388, dtype: int64

In [21]:
# Check processed val set for nullls

X_val_proc.isnull().sum().sort_values(ascending=False)

MasVnrArea        0
ExterQual         0
ExterCond         0
HeatingQC         0
SaleType_ConLw    0
                 ..
BedroomAbvGr      0
3SsnPorch         0
2ndFlrSF          0
1stFlrSF          0
Functional        0
Length: 388, dtype: int64

In [22]:
# Import and fit to linear regression model

from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train_proc, log_y_train)

LinearRegression()

In [23]:
# Read test CSV and check header

test_data = pd.read_csv('../Data and Notes/test.csv')
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [24]:
# Submission needs ID column, split off to recombine later

id_col = test_data.loc[:,'Id']
id_col = pd.DataFrame(id_col)

In [25]:
# Transform test data and check header

test_proc = main_pipe.transform(test_data)
test_proc.head()

,ExterQual,ExterCond,HeatingQC,KitchenQual,Functional,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,Alley_Grvl,Alley_NA,Alley_Pave,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NA,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NA,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NA,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NA,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NA,BsmtFinType2_Rec,BsmtFinType2_Unf,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NA,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageYrBlt_1900.0,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935.0,GarageYrBlt_1936.0,GarageYrBlt_1937.0,GarageYrBlt_1938.0,GarageYrBlt_1939.0,GarageYrBlt_1940.0,GarageYrBlt_1941.0,GarageYrBlt_1942.0,GarageYrBlt_1945.0,GarageYrBlt_1946.0,GarageYrBlt_1948.0,GarageYrBlt_1949.0,GarageYrBlt_1950.0,GarageYrBlt_1951.0,GarageYrBlt_1952.0,GarageYrBlt_1953.0,GarageYrBlt_1954.0,GarageYrBlt_1955.0,GarageYrBlt_1956.0,GarageYrBlt_1957.0,GarageYrBlt_1958.0,GarageYrBlt_1959.0,GarageYrBlt_1960.0,GarageYrBlt_1961.0,GarageYrBlt_1962.0,GarageYrBlt_1963.0,GarageYrBlt_1964.0,GarageYrBlt_1965.0,GarageYrBlt_1966.0,GarageYrBlt_1967.0,GarageYrBlt_1968.0,GarageYrBlt_1969.0,GarageYrBlt_1970.0,GarageYrBlt_1971.0,GarageYrBlt_1972.0,GarageYrBlt_1973.0,GarageYrBlt_1974.0,GarageYrBlt_1975.0,GarageYrBlt_1976.0,GarageYrBlt_1977.0,GarageYrBlt_1978.0,GarageYrBlt_1979.0,GarageYrBlt_1980.0,GarageYrBlt_1981.0,GarageYrBlt_1982.0,GarageYrBlt_1983.0,GarageYrBlt_1984.0,GarageYrBlt_1985.0,GarageYrBlt_1986.0,GarageYrBlt_1987.0,GarageYrBlt_1988.0,GarageYrBlt_1989.0,GarageYrBlt_1990.0,GarageYrBlt_1991.0,GarageYrBlt_1992.0,GarageYrBlt_1993.0,GarageYrBlt_1994.0,GarageYrBlt_1995.0,GarageYrBlt_1996.0,GarageYrBlt_1997.0,GarageYrBlt_1998.0,GarageYrBlt_1999.0,GarageYrBlt_2000.0,GarageYrBlt_2001.0,GarageYrBlt_2002.0,GarageYrBlt_2003.0,GarageYrBlt_2004.0,GarageYrBlt_2005.0,GarageYrBlt_2006.0,GarageYrBlt_2007.0,GarageYrBlt_2008.0,GarageYrBlt_2009.0,GarageYrBlt_2010.0,GarageYrBlt_nan,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_NA,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_NA,GarageCond_Po,GarageCond_TA,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_NA,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_NA,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighbor

In [26]:
# Predict logged target

log_test_pred = lr.predict(test_proc)
log_test_pred = pd.DataFrame(log_test_pred)

In [27]:
# Anti-log target for submission

test_pred = np.exp(log_test_pred)

In [28]:
# Add ID column back for submission

submit = id_col.join(test_pred)
submit.columns = ['Id', 'SalePrice']
submit

,Id,SalePrice
0,1461,115850.733540
1,1462,149203.837668
2,1463,184065.995167
3,1464,203907.515392
4,1465,190523.111787
...,...,...
1454,2915,79579.656452
1455,2916,79056.353658
1456,2917,169827.301999
1457,2918,114027.955552


In [29]:
# Export to CSV for submission

submit.to_csv('test_predicts_final.csv', index=False)

In [30]:
# Import Decision Tree

from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()

In [31]:
# Fit model

dt.fit(X_train_proc, log_y_train)

DecisionTreeRegressor()

In [32]:
# Predict model

dt_log_test_pred = dt.predict(test_proc)
dt_log_test_pred = pd.DataFrame(dt_log_test_pred)

In [33]:
# anti-log predictions

dt_test_pred = np.exp(dt_log_test_pred)

In [34]:
# rejoin columns for submission

dt_submit = id_col.join(dt_test_pred)
dt_submit.columns = ['Id', 'SalePrice']
dt_submit

,Id,SalePrice
0,1461,144000.0
1,1462,207500.0
2,1463,187000.0
3,1464,187000.0
4,1465,189000.0
...,...,...
1454,2915,37900.0
1455,2916,83500.0
1456,2917,159000.0
1457,2918,93500.0


In [35]:
# Export for submission

dt_submit.to_csv('dt_predicts_final.csv', index=False)

In [36]:
# Functions for metrics to evaluate regression

from sklearn.metrics import (mean_absolute_error, mean_squared_error, root_mean_squared_error,
r2_score, mean_absolute_percentage_error)

def adj_r2(r2, x):
  n = x.shape[0]
  p = x.shape[1]
  return 1 - (((n-1) / (n - p- 1)) * (1 - r2))

def evaluate_regression(model, X, y, name='model'):
  y_pred = model.predict(X)
  MAPE = mean_absolute_percentage_error(y, y_pred)
  r2 = r2_score(y, y_pred)
  RMSE = root_mean_squared_error(y, y_pred)
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  a_r2 = adj_r2(r2, X)
  metrics = ['MAE','MSE','RMSE','MAPE','R2','adj_r2']
  results = pd.DataFrame(columns=metrics, index=[name])
  results['MAE'] = [MAE]
  results['MSE'] = [MSE]
  results['RMSE'] = RMSE
  results['MAPE'] = MAPE
  results['R2'] = r2
  results['adj_r2'] = a_r2

  return results


In [37]:
# Check default depth

dt.get_depth()

23

In [ ]:
# Create results Dataframe with evaluator function

val_results = evaluate_regression(lr, X_val_proc, log_y_val, 'lin reg')
val_results

In [49]:
# Add resutls of default depth decision tree

val_results = pd.concat([val_results, evaluate_regression(dt, X_val_proc, log_y_val, 'no_max_depth_tree')])
val_results

In [54]:
# Decision tree max depth 4 fit

dt2 = DecisionTreeRegressor(max_depth=4)
dt2.fit(X_val_proc, log_y_val)

DecisionTreeRegressor(max_depth=4)

In [55]:
# Add results of max depth 4 decision tree to results table

val_results = pd.concat([val_results, evaluate_regression(dt2, X_val_proc, log_y_val, 'dt_max_4')])
val_results

,MAE,MSE,RMSE,MAPE,R2,adj_r2
no_max_depth_tree,0.162066,0.049185,0.221777,0.013573,0.677413,1.954595
lin reg,0.101621,0.041473,0.203649,0.008530,0.727994,1.804915
dt_max_4,0.117026,0.024560,0.156717,0.009776,0.838918,1.476670


In [56]:
# Decision tree max depth 4 fit


dt_dep_4_preds = dt2.predict(test_proc)
dt_dep_4_preds = pd.DataFrame(dt_dep_4_preds)

In [57]:
# Anti-log

dt4_test_preds = np.exp(dt_dep_4_preds)

In [58]:
# Make for submission

dt4depth_submit = id_col.join(dt4_test_preds)
dt4depth_submit.columns = ['Id', 'SalePrice']
dt4depth_submit


,Id,SalePrice
0,1461,121180.404288
1,1462,149503.429035
2,1463,170943.227815
3,1464,170943.227815
4,1465,195958.782276
...,...,...
1454,2915,97562.640068
1455,2916,97562.640068
1456,2917,149503.429035
1457,2918,121180.404288


In [59]:
dt4depth_submit.to_csv('dt4depth_predicts.csv', index=False)

In [60]:
dt3 = DecisionTreeRegressor(max_depth=8)
dt3.fit(X_val_proc, log_y_val)

DecisionTreeRegressor(max_depth=8)

In [61]:
val_results = pd.concat([val_results, evaluate_regression(dt3, X_val_proc, log_y_val, 'dt_max_8')])
val_results

,MAE,MSE,RMSE,MAPE,R2,adj_r2
no_max_depth_tree,0.162066,0.049185,0.221777,0.013573,0.677413,1.954595
lin reg,0.101621,0.041473,0.203649,0.008530,0.727994,1.804915
dt_max_4,0.117026,0.024560,0.156717,0.009776,0.838918,1.476670
dt_max_8,0.034429,0.003389,0.058218,0.002895,0.977771,1.065781


In [62]:
dt_dep_8_preds = dt3.predict(test_proc)
dt_dep_8_preds = pd.DataFrame(dt_dep_8_preds)

In [63]:
dt8_test_preds = np.exp(dt_dep_8_preds)

In [64]:
dt8depth_submit = id_col.join(dt8_test_preds)
dt8depth_submit.columns = ['Id', 'SalePrice']
dt8depth_submit


,Id,SalePrice
0,1461,121611.969273
1,1462,141249.291845
2,1463,165600.000000
3,1464,165600.000000
4,1465,239699.916562
...,...,...
1454,2915,84355.197070
1455,2916,84355.197070
1456,2917,141249.291845
1457,2918,121611.969273


In [65]:
dt8depth_submit.to_csv('dt8depth_predicts.csv', index=False)

In [67]:
# Import Grid search

from sklearn.model_selection import GridSearchCV

# Model
dec_tree = DecisionTreeRegressor(random_state=42)

# Setup max depth hyper parameter
param_grid = {'max_depth': list(range(4, 61, 2))}

# Setup gridsearch
grid_search = GridSearchCV(estimator=dec_tree, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit data
grid_search.fit(X_train_proc, log_y_train)

# depth check
best_depth = grid_search.best_params_['max_depth']
best_depth

Best depth for the DecisionTreeRegressor: 6


In [68]:
dt4 = DecisionTreeRegressor(max_depth=6)
dt4.fit(X_train_proc, log_y_train)

DecisionTreeRegressor(max_depth=6)

In [69]:
val_results = pd.concat([val_results, evaluate_regression(dt4, X_val_proc, log_y_val, 'dt_max_6')])
val_results

,MAE,MSE,RMSE,MAPE,R2,adj_r2
no_max_depth_tree,0.162066,0.049185,0.221777,0.013573,0.677413,1.954595
lin reg,0.101621,0.041473,0.203649,0.008530,0.727994,1.804915
dt_max_4,0.117026,0.024560,0.156717,0.009776,0.838918,1.476670
dt_max_8,0.034429,0.003389,0.058218,0.002895,0.977771,1.065781
dt_max_6,0.154449,0.046407,0.215423,0.012956,0.695631,1.900683


In [70]:
depth6_preds = dt4.predict(test_proc)
depth6_preds = pd.DataFrame(depth6_preds)

In [71]:
d6_test_preds = np.exp(depth6_preds)

In [74]:
d6_sumbit = id_col.join(d6_test_preds)
d6_sumbit.columns = ['Id', 'SalePrice']
d6_sumbit

,Id,SalePrice
0,1461,135100.293660
1,1462,189967.197400
2,1463,191847.238456
3,1464,191847.238456
4,1465,189127.311861
...,...,...
1454,2915,79135.958957
1455,2916,79135.958957
1456,2917,163790.079794
1457,2918,96179.283796


In [75]:
d6_sumbit.to_csv('dt6depth_predicts.csv', index=False)